In [1]:
/* Import dataset */
filename injury url "https://raw.githubusercontent.com/rachelnisbet/SAS-Procs-Oh-My/main/injury_data.csv";

proc import datafile=injury
    out=injury_data dbms= csv replace;
run;

SAS server started using Context SAS Studio compute context with SESSION_ID=5260ad52-778a-4f8f-ae52-213c5504034d-ses0000
12   ods listing close;ods html5 (id=saspy_internal) options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
12 ! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: sashtml.htm
13   
14   /* Import dataset */
15   filename injury url "https://raw.githubusercontent.com/rachelnisbet/SAS-Procs-Oh-My/main/injury_data.csv";
16   
17   proc import datafile=injury
18       out=injury_data dbms= csv replace;
19   run;

NOTE: Unable to open parameter catalog: SASUSER.PARMS.PARMS.SLIST in update mode. Temporary parameter values will be saved to 
WORK.PARMS.PARMS.SLIST.
20    /**********************************************************************
21    *   PRODUCT:   SAS
22    *   VERSION:   V.04.00
23    *   CREATOR:   External File Interface
24    *   DATE:      13MAY24
25    *   DESC:      Generated SAS Datastep Code
26    *   TEMPLATE SOURCE:  

In [5]:
/* PROC SQL: Using SQL queries */
proc sql;
  select *
  from injury_data
  where Player_Age > 35;
quit;

proc sql;
    select previous_injuries, recovery_time, mean(training_intensity) as Avg_Intensity
    from injury_data
    group by previous_injuries, recovery_time;
quit;

Player_Age,Player_Weight,Player_Height,Previous_Injuries,Training_Intensity,Recovery_Time,Likelihood_of_Injury
37,70.996271268,174.58165012,0,0.226521626,6,1
38,75.820548712,206.6318235,1,0.359208747,4,0
36,79.038205587,181.52315514,1,0.8206961608,3,1
38,90.097713125,179.17352215,0,0.3625597966,3,0
39,87.869749638,175.51619781,1,0.0846883419,2,1
38,70.553624418,200.10009289,1,0.4666931209,6,1
38,88.859033213,193.97226423,0,0.8084057389,3,0
39,77.809643209,179.4111819,0,0.4591240233,2,0
36,76.805308418,165.53871967,0,0.0326590608,1,1
37,72.971537803,176.39526596,1,0.407748105,1,0


In [3]:
proc python;
submit;

import pandas as pd

# Read the SAS dataset into a pandas DataFrame
injury_df = pd.read_csv("https://raw.githubusercontent.com/rachelnisbet/SAS-Procs-Oh-My/main/injury_data.csv")


# Filter data where Player_Age > 35 using Python
filtered_injury = injury_df[injury_df['Player_Age'] > 35]

# Print the filtered DataFrame
print(filtered_injury.head())

# Calculate and print summary statistics
summary = injury_df.describe()
print(summary)

# Calculate correlation matrix
correlation_matrix = injury_df.corr()

# Print correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)

# Extract correlation with 'Likelihood_of_Injury' variable
likelihood_correlation = correlation_matrix['Likelihood_of_Injury']

# Print correlation with 'Likelihood_of_Injury'
print("\nCorrelation with Likelihood_of_Injury:")
print(likelihood_correlation)

endsubmit;
run;

67   ods listing close;ods html5 (id=saspy_internal) options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /
67 ! outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: sashtml2.htm
68   
69   proc python;
70   submit
NOTE: Python initialized.
Python 3.9.18 (main, Mar 26 2024, 01:56:30) 
[GCC 8.5.0 20210514 (Red Hat 8.5.0-20)] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>>
>>> 
70 !       ;
71   
72   import pandas as pd
73   
74   # Read the SAS dataset into a pandas DataFrame
75   injury_df = pd.read_csv("https://raw.githubusercontent.com/rachelnisbet/SAS-Procs-Oh-My/main/injury_data.csv")
76   
77   
78   # Filter data where Player_Age > 35 using Python
79   filtered_injury = injury_df[injury_df['Player_Age'] > 35]
80   
81   # Print the filtered DataFrame
82   print(filtered_injury.head())
83   
84   # Calculate and print summary statistics
85   summary = injury_df.describe()
86   print(summary)
87   
88   # Calculate c

In [4]:
/* PROC IML: Using the interactive matrix language */

proc iml;
submit / R;

# Read in the CSV file
injury_data <- read.csv("https://raw.githubusercontent.com/rachelnisbet/SAS-Procs-Oh-My/main/injury_data.csv")

# Filter the data

filtered_data <- injury_data[injury_data$Player_Age >= 35, ]

       

# Print data

print(filtered_data)

 

# Summary statistics

summary_stats <- summary(injury_data)

       

# Print summary statistics

print("Here comes summary stats....")

print(summary_stats)

 

# Define age groups (e.g., 17-25, 26-35, etc.)

age_breaks <- c(17, 25, 35, max(injury_data$Player_Age))

age_labels <- c("18-25", "26-35", "36+")

age_groups <- cut(injury_data$Player_Age, breaks = age_breaks, labels = age_labels)

 

# Add age groups as a new column in the dataset

injury_data$Age_Group <- age_groups

 

# Calculate and print frequencies for Age Group

summary_stats2 <- summary(injury_data$Age_Group)

print("Here comes Age Group freq counts....")

print(summary_stats2)

 

endsubmit;

run;